In [1]:
# coding: utf-8
import numpy as np
import pandas as pd

import random

pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

In [13]:
def elo_prob(rw=1600, rb=1600):
    """
    base elo propability without chancging k for newcommets etc.
    """
    rw = float(rw)
    rb = float(rb)
    res = 1 / (1 + np.power(10, (rb-rw) / 400))
    return res

def get_match(rating1=1600, rating2=1600, total=0, sigma=2.5):
    """ 
    get match result according elo rating, total scores and sigma of probability distribution
    """
    import random
    import math

    # base probability
    # based on 100 elo meand approximately win on 1 goal (1-0, 2-1 etc.)
    norn_factor = 15
    mult_factor = 100
    mu = mult_factor * (elo_prob(rating1, rating2) - 0.5) / norn_factor

    # current different between teams in this proba
    base_res = round(random.normalvariate(mu, sigma), 0)

    # scores from total and diffs
    if base_res > 0:
        res2 = total
        res1 = total + base_res
    else:
        res1 = total
        res2 = total - base_res


    return int(res1), int(res2)


def get_round_robin_schedule(teams:list, num_cycles=2):
    """ 
    get round robin schedule for teams
    """ 
    if len(teams) % 2 != 0:
        teams.append(None)
    
    num_rounds = len(teams) - 1
    schedule = []
    
    for cycle in range(num_cycles):
        matches = []
        for round in range(num_rounds):
            for i in range(len(teams)//2):
                if teams[i] is None or teams[len(teams)-1-i] is None:
                    continue

                matches.append((teams[i], teams[len(teams)-1-i]))
            
            teams = [teams[0]] + [teams[-1]] + teams[1:len(teams)-1]
        
        if cycle != 0:
            invert_m = []
            for m in matches:
                invert_m.append((m[1], m[0]))
            matches = invert_m
        
        schedule.append(matches)
    
    return schedule

def get_roudn_robin_match_results(teams_dct, num_cycles=2):
    """
    get round robin match results for teams
    """
    teams_lst = list(teams_dct.keys())
    round_robin_schedule_lst = get_round_robin_schedule(teams_lst, num_cycles)

    id1 = []
    id2 = []
    team1 = []
    team2 = []
    sc1 = []
    sc2 = []
    for round in round_robin_schedule_lst:
        for pair in round:
            total = round(random.uniform(0, 3),0)
            sigma = round(random.uniform(2, 3),2)
            res = get_match(
                        teams_dct[pair[0]][1], 
                        teams_dct[pair[1]][1],
                        total, sigma
                            )
            id1.append(pair[0])
            id2.append(pair[1])
            team1.append(teams_dct[pair[0]][0])
            team2.append(teams_dct[pair[1]][0])
            sc1.append(res[0])
            sc2.append(res[1])
    res_df = pd.DataFrame([id1, id2, team1, team2, sc1, sc2]).T
    res_df.columns = ['id1', 'id2', 'team1', 'team2', 'score1', 'score2']

    return res_df

def get_round_robin_table(teams_dct, total=0, sigma=2.5, num_cycles=2):
    """
    get round robin tournament table for teams
    """

    res_df = get_roudn_robin_match_results(teams_dct, num_cycles)
    res_df['points1'] = np.where(
        res_df['score1'] > res_df['score2'], 3,
        np.where(res_df['score1'] < res_df['score2'], 0, 1)
                                )
    res_df['points2'] = np.where(
        res_df['score1'] > res_df['score2'], 0,
        np.where(res_df['score1'] < res_df['score2'], 3, 1)
                                )
    
    res_df['games'] = 1
    
    tournament_results_df_home = res_df.groupby(['id1', 'team1']).agg(
                        {'score1': 'sum', 'points1': 'sum', 'games': 'sum'}
                                                                ).reset_index()
    tournament_results_df_away = res_df.groupby(['id2', 'team2']).agg(
                        {'score2': 'sum', 'points2': 'sum', 'games': 'sum'}
                                                                ).reset_index()
    tournament_results_df_home.columns = ['id', 'team', 'score', 'points', 'games']
    tournament_results_df_away.columns = ['id', 'team', 'score', 'points', 'games']

    
    tournament_results_df = tournament_results_df_home.merge(tournament_results_df_away, 'left', on=['id'], suffixes=['_home', '_away'])
    tournament_results_df['points'] = tournament_results_df['points_home'] + tournament_results_df['points_away']
    tournament_results_df['scored'] = tournament_results_df['score_home'] + tournament_results_df['score_away']
    tournament_results_df['games'] = tournament_results_df['games_home'] + tournament_results_df['games_away']

    tournament_results_df = tournament_results_df.sort_values(by=['points', 'scored'], ascending=False)

    tournament_results_df = tournament_results_df.rename(columns={'team_home': 'team'})

    tournament_results_df = tournament_results_df[[
        'id', 'team', 'games', 'points', 'scored', 'points_home', 'score_home', 'games_home', 'points_away', 'score_away', 'games_away'
    ]]
    
    return tournament_results_df, res_df

In [14]:
# small example teamset
# 4 pairs of teams with approximately same rating
# with huge gap between each other
teams_dct = {
    1: ['Barcelona', 2630, 1],
    2: ['Real Madrid', 2600, 1],
    3: ['Osasuna', 2420, 1],
    4: ['Espanyol', 2330, 1],
    5: ['Tenerife', 1800, 1],
    6: ['Zaragoza', 1750, 1],
    7: ['Ronaldo fun club', 1280, 1],
    8: ['Messi fun club', 1200, 1],
            }

In [9]:
get_match(1000, 900, 0, 2.5)

(1, 0)

In [15]:
tournament_results_df, res_df = get_round_robin_table(teams_dct)
tournament_results_df

,id,team,games,points,scored,points_home,score_home,games_home,points_away,score_away,games_away
1,2,Real Madrid,14,36,44,18,19,7,18,25,7
0,1,Barcelona,14,33,30,17,22,7,16,8,7
2,3,Osasuna,14,32,40,18,22,7,14,18,7
3,4,Espanyol,14,23,28,11,12,7,12,16,7
4,5,Tenerife,14,18,15,9,9,7,9,6,7
5,6,Zaragoza,14,12,12,6,2,7,6,10,7
6,7,Ronaldo fun club,14,7,6,0,0,7,7,6,7
7,8,Messi fun club,14,3,4,0,0,7,3,4,7


In [64]:

total = round(random.uniform(0, 3),0)
sigma = round(random.uniform(2, 3),2)
sigma

2.51